In [15]:
import os
import cv2
import numpy as np
import pandas as pd
import copy
from itertools import zip_longest
import sys
import warnings
from google.colab.patches import cv2_imshow

warnings.simplefilter(action='ignore', category=FutureWarning)

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/CV/data'
filename = '_.avi'

### Each video shows one person performing the same movements at distances of 3.5, 5.5 and 7.5 meters.

Note: Due to private data, the images displayed have been removed

In [46]:
def get_frame(cap, frame_number):
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number < 0 or frame_number >= total_frames:
        raise Exception('Incorrect input data, frame number must be in range [0, '+str(int(total_frames-1))+']')
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        _, frame = cap.read()
        return frame, total_frames

cap = cv2.VideoCapture(os.path.join(data_dir, '2022-04-13 16-29 L.avi'))
frame, total_frames = get_frame(cap, 20)

In [ ]:
def get_frame(cap, frame_number):
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number < 0 or frame_number >= total_frames:
        raise Exception('Incorrect input data, frame number must be in range [0, '+str(int(total_frames-1))+']')
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        _, frame = cap.read()
        return frame, total_frames

cap = cv2.VideoCapture(os.path.join(data_dir, filename))
frame, total_frames = get_frame(cap, 100)

cap = cv2.VideoCapture(os.path.join(data_dir, '2022-04-13 16-29 L.avi'))
frame5, total_frames = get_frame(cap, 100)

cap = cv2.VideoCapture(os.path.join(data_dir, '2022-04-13 16-36 L.avi'))
frame7, total_frames = get_frame(cap, 100)

cv2_imshow(np.hstack([frame, frame5, frame7]))

### Wczytanie pliku z oznaczeniami

In [19]:
df = pd.read_csv(os.path.join(data_dir, '2022-04-13 16-01 L.csv'))
df

,frame,right foot,right hand,head,left hand,left foot
0,0,"(668, 818)","(670, 603)","(764, 416)","(845, 595)","(760, 808)"
1,1,"(662, 819)","(665, 607)","(760, 414)","(844, 596)","(758, 807)"
2,2,"(659, 820)","(657, 612)","(752, 408)","(840, 593)","(765, 811)"
3,3,"(658, 829)","(655, 611)","(739, 414)","(844, 590)","(755, 813)"
4,4,"(647, 817)","(645, 604)","(725, 411)","(842, 578)","(756, 811)"
...,...,...,...,...,...,...
199,199,"(647, 823)","(623, 609)","(716, 397)","(814, 573)","(742, 811)"
200,200,"(648, 813)","(623, 602)","(722, 397)","(818, 574)","(742, 812)"
201,201,"(648, 823)","(638, 602)","(730, 404)","(826, 578)","(746, 810)"
202,202,"(653, 823)","(644, 597)","(742, 406)","(836, 583)","(734, 813)"


# Implementation of body shaping methods

## OpenPose 
https://colab.research.google.com/github/Kasidit0052/MatchingOpenpose/blob/main/OpenPose.ipynb#scrollTo=_RjdkypNxXu6

### By default the method returns 25 silhouette points, for the purpose of the task only 5 points were taken.

In [ ]:
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
!mkdir openpose/images openpose/videos 

In [ ]:
OpenposeDir = '/content/openpose/'
# Set Python Openpose Directory for python api (Important)
pyopenpose_dir = os.path.join(OpenposeDir,'build','python') # ex: '/content/openpose/build/python'
if pyopenpose_dir not in sys.path:
    sys.path.append(pyopenpose_dir)
from openpose import pyopenpose as op

# Custom Params (refer to openpose/include/openpose/flags.hpp for more parameters)
params = dict()
params["model_folder"] = os.path.join(OpenposeDir,'models')  # ex: '/content/openpose/models'

# Starting OpenPose
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    args = [iter(iterable)] * n
    return list(zip_longest(*args, fillvalue=fillvalue))

def get_coors(frame, frame_num):
  datum = op.Datum()
  datum.cvInputData = frame
  opWrapper.emplaceAndPop(op.VectorDatum([datum]))
  network_output = datum.poseKeypoints

  BODY_PARTS = op.getPoseBodyPartMapping(op.BODY_25)
  POSE_PAIRS = op.getPosePartPairs(op.BODY_25)
  POSE_PAIRS = grouper(POSE_PAIRS, 2, None)

  index_names = copy.deepcopy(BODY_PARTS)
  index_names.pop(25)
  for human in network_output:
      output_sub_arr = {}
      for i in range(len(human)):
          output_sub_arr[BODY_PARTS[i]]= [int(human[i][0]),int(human[i][1])]
      df = pd.DataFrame(output_sub_arr.values(), columns=['x','y'], index=list(index_names.values()))
      res = np.array(df.loc[df.index.isin(['RWrist', 'RHeel', 'LHeel', 'Nose', 'LWrist', 'LHeel'])])
      res = list(map(tuple, np.take(res, [1, -1, 0, 2, -2], axis=0)))
      res = [frame_num]+res
      return res

frame = get_frame(cap, 200)
get_coors(frame, 200)

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/wk/data'
filename = '2022-04-13 16-29 L.avi'
cap = cv2.VideoCapture(os.path.join(data_dir, filename))
frame_number = 0
openpose_res = []

while True:
  ret, frame = cap.read()
  if not ret:
    break
  frame_number += 1
  openpose_res.append(get_coors(frame, frame_number))

openpose_df = pd.DataFrame(openpose_res, columns=['frame', 'right foot', 'right hand', 'head', 'left hand', 'left foot'])
openpose_df.to_csv(os.path.join(data_dir, 'openpose 2022-04-13 16-36 L.csv'), index=False)

In [ ]:
# Process Image
datum = op.Datum()
# input_image = cv2.imread(images_list[0]) # Change Image Here
datum.cvInputData = frame
opWrapper.emplaceAndPop(op.VectorDatum([datum]))
network_output = datum.poseKeypoints

# Display Image
cv2_imshow(datum.cvOutputData)

## AlphaPose 
https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/AlphaPose.ipynb#scrollTo=G9pi9T2t_3fR

### Domyślnie metoda zwraca 17 punktów sylwetki, na potrzebę zadania wzięto tylko 5 punków.

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q -b pytorch --depth 1 $git_repo_url
  !cd $project_name && pip install -q -r requirements.txt
  !pip install -q youtube-dl visdom
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

In [ ]:
! pip install pyyaml==5.2
! pip install scipy==1.1.0
! pip install torch==1.2.0 torchvision==0.4.0
! pip install pillow==6.2.2

In [ ]:
import torch
print(torch.__version__)
import yaml, scipy
print(yaml.__version__)
print(scipy.__version__)

In [ ]:
!rm -rf /content/AlphaPose
os.chdir('/content/')
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

Cloning into 'AlphaPose'...
remote: Enumerating objects: 2301, done.
remote: Total 2301 (delta 0), reused 0 (delta 0), pack-reused 2301
Receiving objects: 100% (2301/2301), 108.65 MiB | 15.23 MiB/s, done.
Resolving deltas: 100% (1104/1104), done.


In [ ]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

In [ ]:
import os
os.chdir('/content/AlphaPose')
print(os.getcwd())
! python setup.py build develop

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

! mkdir /content/AlphaPose/detector/yolo/data
file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/yolo/data/yolov3-spp.weights')

! mkdir /content/AlphaPose/detector/tracker/data
file_id = '1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty')

file_id = '1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('/content/AlphaPose/pretrained_models/fast_res50_256x192.pth')

In [ ]:
!pip install cython-bbox

In [ ]:
import cv2
def save_jpg(data_dir, filename):
  cap = cv2.VideoCapture(os.path.join(data_dir, filename))
  frame_number = 0
  while True:
    ret, frame = cap.read()
    if not ret:
      break
    cv2.imwrite('/content/drive/MyDrive/lab10/'+str(frame_number)+'.jpg', frame)
    frame_number += 1

data_dir = '/content/drive/MyDrive/Colab Notebooks/wk/data'
filename = '2022-04-13 16-36 L.avi'
save_jpg(data_dir, filename)

In [ ]:
import os
os.chdir('/content/AlphaPose')

! python3 scripts/demo_inference.py --cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml --checkpoint pretrained_models/fast_res50_256x192.pth --indir /content/drive/MyDrive/lab10
# result json and rendered images are saved here:
! ls examples/res/

In [ ]:
import json
import numpy as np
import pandas as pd

data_dir = '/content/drive/MyDrive/Colab Notebooks/wk/data'
results = []
idx = 0
with open('/content/AlphaPose/examples/res/alphapose-results.json') as json_file:
  data = json.load(json_file)
  if data:
    for idata in data:
      keypoints = idata['keypoints']
      x = np.array(keypoints[0::3])
      y = np.array(keypoints[1::3])
      x = x[[15, 9, 0, 10, 16]]
      y = y[[15, 9, 0, 10, 16]]
      coors = list(map(tuple, np.vstack((x, y)).T))
      res = [idx]
      res += coors
      results.append(res)
      idx += 1

In [ ]:
alphapose_df = pd.DataFrame(results, columns=['frame', 'right foot', 'right hand', 'head', 'left hand', 'left foot'])
alphapose_df.to_csv(os.path.join(data_dir, 'alphapose 2022-04-13 16-36 L.csv'), index=False)

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/wk/data/104.jpg')
cv2_imshow(img)

## Mediapipe 
https://google.github.io/mediapipe/solutions/pose

### By default the method returns 32 silhouette points, for the purpose of the task only 5 points were taken.

In [ ]:
!pip install mediapipe

In [ ]:
import mediapipe as mp
import copy

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def get_coors(frame, frame_num):
  row = [frame_num]
  with mp_pose.Pose(
      static_image_mode=True,
      model_complexity=2,
      enable_segmentation=True,
      min_detection_confidence=0.5) as pose:

      image_height, image_width, _ = frame.shape
      results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
      if results.pose_landmarks:
        res = results.pose_landmarks.landmark
        for idx in [30, 16, 0, 15, 29]:
          if res[idx]:
            row.append((int(image_width * res[idx].x), int(image_height * res[idx].y)))
      else:
        row.append([np.full(5, np.nan)])
  return row

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/wk/data'
filename = '2022-04-13 16-36 L.avi'
cap = cv2.VideoCapture(os.path.join(data_dir, filename))
frame_number = 0
mediapipe_res = []

while True:
  ret, frame = cap.read()
  if not ret:
    break
  frame_number += 1
  mediapipe_res.append(get_coors(frame, frame_number))

mediapipe_df = pd.DataFrame(mediapipe_res, columns=['frame', 'right foot', 'right hand', 'head', 'left hand', 'left foot'])
mediapipe_df.to_csv(os.path.join(data_dir, 'mediapipe 2022-04-13 16-36 L.csv'), index=False)

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:
    image = frame
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    annotated_image = image.copy()
    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
    annotated_image = np.where(condition, annotated_image, copy.copy(frame))
    # Draw pose landmarks on the image.
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    cv2_imshow(annotated_image)

## Multi-Person Pose Estimation model

### The method returns 44 points, the least likely points can be discarded by setting an appropriate threshold.

In [31]:
protoFile = '/content/drive/MyDrive/Colab Notebooks/wk/pose_deploy_linevec_faster_4_stages.prototxt'
weightsFile = '/content/drive/MyDrive/Colab Notebooks/wk/pose_iter_160000.caffemodel'
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [ ]:
def get_coors(net, frame, frame_num):
  inHeight, inWidth, _ = frame.shape
  inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
  net.setInput(inpBlob)
  output = net.forward()
  row = [frame_num]
  H = output.shape[2]
  W = output.shape[3]
  for i in [10, 4, 0, 7, 13]:
      probMap = output[0, i, :, :]
      minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
      x = (inWidth * point[0]) / W
      y = (inHeight * point[1]) / H
      if prob > 0.1 :
          row.append((int(x), int(y)))
      else :
          row.append(np.nan)
  return row

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/wk/data'
filename = '2022-04-13 16-36 L.avi'
cap = cv2.VideoCapture(os.path.join(data_dir, filename))
frame_number = 0
mpii_res = []

ret, frame = cap.read()
mpii_res.append(get_coors(net, frame, frame_number))
mpii_df = pd.DataFrame(mpii_res, columns=['frame', 'right foot', 'right hand', 'head', 'left hand', 'left foot'])
mpii_df.to_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-36 L.csv'), index=False, mode='w')

while True:
  mpii_res = []
  ret, frame = cap.read()
  if not ret:
    break
  frame_number += 1
  mpii_res.append(get_coors(net, frame, frame_number))
  mpii_df = pd.DataFrame(mpii_res, columns=['frame', 'right foot', 'right hand', 'head', 'left hand', 'left foot'])
  mpii_df.to_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-36 L.csv'), index=False, header=False, mode='a')
cap.release()

In [ ]:
mpii_df = pd.read_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-36 L.csv'))
mpii_df = mpii_df.fillna('(0, 0)')
mpii_df.to_csv(os.path.join(data_dir, 'm2pii 2022-04-13 16-36 L.csv'), index=False, mode='w')

In [50]:
inHeight, inWidth, _ = frame5.shape
inpBlob = cv2.dnn.blobFromImage(frame5, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
net.setInput(inpBlob)
output = net.forward()
H = output.shape[2]
W = output.shape[3]

In [ ]:
image = copy.deepcopy(frame5)
for i in range(output.shape[1]):
    probMap = output[0, i, :, :]
    minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
    x = (inWidth * point[0]) / W
    y = (inHeight * point[1]) / H
    if prob > 0.6:
      cv2.circle(image, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
      cv2.putText(image, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)
cv2_imshow(image)

# Implementation of metrics to compare methods

## Percentage of Detected Joints

In [109]:
def pdj_calc(df_orig, df_calc, frame, total_frames, param):
  diagonal = lambda length, width: np.sqrt(length**2 + width** 2)
  pdj_i = lambda orig, calc, diag: np.linalg.norm(orig-calc) < diag * param
  bb_size = lambda array: [(max(array[:, 0]) - min(array[:, 0])), (max(array[:, 1]) - min(array[:, 1]))]
  pdj_res = 0

  for (idxRow, r1), (_, r2) in zip(df_orig.iloc[:, 1:].iterrows(), df_calc.iloc[:, 1:].iterrows()):
    length, width = bb_size(np.array(list(map(list, r1.to_numpy()))))
    d = diagonal(length, width)
    for idx, ((idxCol, v1), (_, v2)) in enumerate(zip(r1.iteritems(), r2.iteritems())):
      pdj_res += pdj_i(np.array(v1), np.array(v2), d) / (df_orig.shape[1]-1)
  return pdj_res / total_frames

def create_table_pdj(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df, param):
  pdj_res = []
  pdj_res.append(pdj_calc(original_df, mediapipe_df, frame, total_frames, param))
  pdj_res.append(pdj_calc(original_df, openpose_df, frame, total_frames, param))
  pdj_res.append(pdj_calc(original_df, mpii_df, frame, total_frames, param))
  pdj_res.append(pdj_calc(original_df, alphapose_df, frame, total_frames, param))

  pdj_df = pd.DataFrame(pdj_res, columns=['score'], 
                        index=['Mediapipe', 'Openpose', 'MPII', 'Alphapose'])
  return pdj_df

## Object Keypoint Similarity

In [94]:
def oks_calc(df_orig, df_calc, frame, total_frames):
  oks = lambda orig, calc, scale, ik: np.exp(-np.linalg.norm(orig-calc) / 2 * scale ** 2 * ik)
  k = [0.089, 0.062, 0.026, 0.062, 0.089]
  area = lambda array: (max(array[:, 0]) - min(array[:, 0])) * (max(array[:, 1]) - min(array[:, 1]))
  image_height, image_width, _ = frame.shape
  total_area = image_height * image_width
  oks_res = np.zeros((df_orig.shape[1]-1))

  for (idxRow, r1), (_, r2) in zip(df_orig.iloc[:, 1:].iterrows(), df_calc.iloc[:, 1:].iterrows()):
    scale = area(np.array(list(map(list, r1.to_numpy())))) / total_area
    for idx, ((idxCol, v1), (_, v2)) in enumerate(zip(r1.iteritems(), r2.iteritems())):
      oks_res[idx] += (oks(np.array(v1), np.array(v2), scale, k[idx]))
  return oks_res / total_frames

def create_table_oks(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df):

  oks_res = []
  oks_res.append(oks_calc(original_df, mediapipe_df, frame, total_frames))
  oks_res.append(oks_calc(original_df, openpose_df, frame, total_frames))
  oks_res.append(oks_calc(original_df, mpii_df, frame, total_frames))
  oks_res.append(oks_calc(original_df, alphapose_df, frame, total_frames))

  oks_df = pd.DataFrame(oks_res, columns=['right foot', 'right hand', 'head', 'left hand', 'left foot'], 
                        index=['Mediapipe', 'Openpose', 'MPII', 'Alphapose'])
  return oks_df

# Method comparison for the first video sequence

In [4]:
import os
import ast

def read_res(avi_name):
  filename = avi_name+'.avi'
  cap = cv2.VideoCapture(os.path.join(data_dir, filename))
  total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  cap.release()

  openpose_df = pd.read_csv(os.path.join(data_dir, 'openpose '+avi_name+'.csv'), 
                            converters={"right foot": ast.literal_eval, 
                                        "right hand": ast.literal_eval,
                                        "head": ast.literal_eval, 
                                        "left hand": ast.literal_eval, 
                                        "left foot": ast.literal_eval
                                        }
                            )	 	 	 	
  mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe '+avi_name+'.csv'),
                            converters={"right foot": ast.literal_eval, 
                                        "right hand": ast.literal_eval,
                                        "head": ast.literal_eval, 
                                        "left hand": ast.literal_eval, 
                                        "left foot": ast.literal_eval
                                        }
                            )

  mpii_df = pd.read_csv(os.path.join(data_dir, 'm2pii '+avi_name+'.csv'), 
                            converters={"right foot": ast.literal_eval, 
                                        "right hand": ast.literal_eval,
                                        "head": ast.literal_eval, 
                                        "left hand": ast.literal_eval, 
                                        "left foot": ast.literal_eval
                                        }
                            )

  alphapose_df = pd.read_csv(os.path.join(data_dir, 'alphapose '+avi_name+'.csv'),
                            converters={"right foot": ast.literal_eval, 
                                        "right hand": ast.literal_eval,
                                        "head": ast.literal_eval, 
                                        "left hand": ast.literal_eval, 
                                        "left foot": ast.literal_eval
                                        }
                            )

  original_df = pd.read_csv(os.path.join(data_dir, avi_name+'.csv'),
                            converters={"right foot": ast.literal_eval, 
                                        "right hand": ast.literal_eval,
                                        "head": ast.literal_eval, 
                                        "left hand": ast.literal_eval, 
                                        "left foot": ast.literal_eval
                                        }
                            )
  return total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df

total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df = read_res('2022-04-13 16-01 L')

### Table of the number of undetected points for all frames

In [85]:
openpose_df = pd.read_csv(os.path.join(data_dir, 'openpose 2022-04-13 16-01 L.csv'))
alphapose_df = pd.read_csv(os.path.join(data_dir, 'alphapose 2022-04-13 16-01 L.csv'))
mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe 2022-04-13 16-01 L.csv'))
mpii_df = pd.read_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-01 L.csv'))

df_na = pd.concat([openpose_df.isna().sum(), 
                 alphapose_df.isna().sum(), 
                 mediapipe_df.isna().sum(),
                 mpii_df.isna().sum()], axis=1)
df_na.rename(columns={0:'Openpose', 2:'Mediapipe', 3:'MPII', 1:'Alphapose'})

,Openpose,Alphapose,Mediapipe,MPII
frame,0,0,0,0
right foot,0,0,0,3
right hand,0,0,0,23
head,0,0,0,0
left hand,0,0,0,20
left foot,0,0,0,8


### PDJ

In [ ]:
create_table_pdj(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df)

,score
Mediapipe,0.854902
Openpose,0.499020
MPII,0.498039
Alphapose,0.194118


### OKS

In [ ]:
create_table_oks(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df)

,right foot,right hand,head,left hand,left foot
Mediapipe,0.996539,0.995615,0.998395,0.995441,0.993811
Openpose,0.904406,0.931080,0.998546,0.994840,0.994104
MPII,0.989982,0.956414,0.997606,0.952132,0.984930
Alphapose,0.972781,0.940695,0.998478,0.940364,0.970550


# Porównanie metod dla filmiku '2022-04-13 16-29 L.avi'

### Table of the number of undetected points for all frames

In [87]:
openpose_df = pd.read_csv(os.path.join(data_dir, 'openpose 2022-04-13 16-29 L.csv'))
alphapose_df = pd.read_csv(os.path.join(data_dir, 'alphapose 2022-04-13 16-29 L.csv'))
mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe 2022-04-13 16-29 L.csv'))
mpii_df = pd.read_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-29 L.csv'))

df_na = pd.concat([openpose_df.isna().sum(), 
                 alphapose_df.isna().sum(), 
                 mediapipe_df.isna().sum(),
                 mpii_df.isna().sum()], axis=1)
df_na.rename(columns={0:'Openpose', 2:'Mediapipe', 3:'MPII', 1:'Alphapose'})

,Openpose,Alphapose,Mediapipe,MPII
frame,0,0,0,0
right foot,0,0,0,0
right hand,0,0,84,10
head,0,0,84,0
left hand,0,0,84,1
left foot,0,0,84,0


In [88]:
mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe '+'2022-04-13 16-29 L'+'.csv'))
mediapipe_df[mediapipe_df == mediapipe_df.iloc[-1, 1]] = np.nan
mediapipe_df = mediapipe_df.fillna('(0, 0)')
mediapipe_df.to_csv(os.path.join(data_dir, 'mediapipe2 2022-04-13 16-29 L.csv'), index=False, mode='w')

### PDJ

In [92]:
total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df = read_res('2022-04-13 16-29 L')
create_table_pdj(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df)

,score
Mediapipe,0.418100
Openpose,0.391855
MPII,0.307692
Alphapose,0.161991


### OKS

In [95]:
create_table_oks(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df)

,right foot,right hand,head,left hand,left foot
Mediapipe,0.984345,0.991220,0.996119,0.989714,0.983548
Openpose,0.984174,0.988727,0.999758,0.999139,0.999028
MPII,0.998665,0.994445,0.999598,0.997922,0.998355
Alphapose,0.996932,0.990761,0.999653,0.990513,0.996602


# Method comparison for the second video sequence

In [97]:
mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe '+'2022-04-13 16-36 L'+'.csv'))
mediapipe_df[mediapipe_df == mediapipe_df.iloc[-1, 1]] = np.nan
mediapipe_df = mediapipe_df.fillna('(0, 0)')
mediapipe_df.to_csv(os.path.join(data_dir, 'mediapipe2 2022-04-13 16-36 L.csv'), index=False, mode='w')

### PDJ

In [112]:
total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df = read_res('2022-04-13 16-36 L')
create_table_pdj(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df, 0.2)

,score
Mediapipe,0.015254
Openpose,0.263559
MPII,0.189831
Alphapose,0.563559


### OKS

In [101]:
create_table_oks(total_frames, original_df, alphapose_df, mpii_df, mediapipe_df, openpose_df)

,right foot,right hand,head,left hand,left foot
Mediapipe,0.987509,0.992728,0.996886,0.991882,0.987159
Openpose,0.933655,0.934691,0.936256,0.935552,0.936201
MPII,0.190627,0.190645,0.190659,0.190626,0.190615
Alphapose,0.999395,0.998751,0.999960,0.998730,0.999359


### Table of the number of undetected points for all frames

In [96]:
openpose_df = pd.read_csv(os.path.join(data_dir, 'openpose 2022-04-13 16-36 L.csv'))
alphapose_df = pd.read_csv(os.path.join(data_dir, 'alphapose 2022-04-13 16-36 L.csv'))
mediapipe_df = pd.read_csv(os.path.join(data_dir, 'mediapipe 2022-04-13 16-36 L.csv'))
mpii_df = pd.read_csv(os.path.join(data_dir, 'mpii 2022-04-13 16-36 L.csv'))

df_na = pd.concat([openpose_df.isna().sum(), 
                 alphapose_df.isna().sum(), 
                 mediapipe_df.isna().sum(),
                 mpii_df.isna().sum()], axis=1)
df_na.rename(columns={0:'Openpose', 2:'Mediapipe', 3:'MPII', 1:'Alphapose'})

,Openpose,Alphapose,Mediapipe,MPII
frame,0,0,0,0
right foot,0,0,0,0
right hand,0,0,232,0
head,0,0,232,0
left hand,0,0,232,0
left foot,0,0,232,0


# Conclusions 
- At longer distances mediapipe fails, missing points are created, although the detected points are still accurate. On the other hand, at shorter distances, it has shown itself to be the most accurate method for silhouette modeling by both ox and pdj measure comparisons.
- The rest of the methods at short distances found points of comparable accuracy, although alphapose performed the worst. 
- At long distances, alphapose scored the best of all the methods.